Idea:

Assume only few spoofing attacks

Synchronize clocks assuming all broadcast locations are correct

Estimate positions for messages based on sensor timestamps

Remove messages with high deviations between broadcast locations and estimated locations

Re-synchronize clocks and re-check broadcast locations

In [1]:
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from collections import defaultdict
from tqdm import tqdm

DATA_LOCATION = "D:/thesis_data/1hour_complete_22_01_10_05/"

sensors = set()
received = defaultdict(lambda: { "sensors": list() })

for file in tqdm(os.listdir(DATA_LOCATION)):
    assert re.match(r"^part-\d{5}$", file)
    with open(os.path.join(DATA_LOCATION, file), "r") as f:
        line = f.readline()
        if not line:
            continue
        assert line == "sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence\n"
        while (line := f.readline().strip()):
            sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence = line.split(',')
            
            if not position_estimator.is_relevant(rawMessage):
                continue

            if 'null' in [sensorLatitude, sensorLongitude, sensorAltitude, timeAtSensor]:
                continue
            
            sensorLatitude = float(sensorLatitude)
            sensorLongitude = float(sensorLongitude)
            sensorAltitude = float(sensorAltitude)
            timeAtSensor = float(timeAtSensor)

            if not (sensorLatitude and sensorLongitude):
                continue

            sensor = (sensorLatitude, sensorLongitude, sensorAltitude, sensorType)
            sensors.add(sensor)
            received[rawMessage]["sensors"].append((timeAtSensor, sensor))

            if not "pos" in received[rawMessage]:
                received[rawMessage]["pos"] = position_estimator.get_pos(rawMessage, timeAtSensor)
    
    break # memory constraint

print("Sensors:", len(sensors))
print("Received Messages:", len(received))

  0%|          | 0/200 [00:23<?, ?it/s]

Sensors: 1364
Received Messages: 77527


In [ ]:
time_delta = defaultdict(list)

for msg in received:
    if not received[msg]["pos"]:
        continue

    for timestamp, sensor in received[msg]["sensors"]:
        
    